# Azure OpenAI 임상 추론 + SOAP 노트 생성

> **Purpose:** Gold Layer의 환자 데이터를 Azure OpenAI API에 전달하여
> SOAP 형식의 진료 가이드를 생성합니다.
> - 환자 바이탈 + 진료 기록 통합 프롬프트 구성
> - Azure OpenAI API 호출 (GPT-5.1 / GPT-4o)
> - 결과를 `P2T2.ai_results.openai_soap_notes` 테이블에 저장
>
> **카탈로그:** `P2T2`
> **스키마:** `ai_results`

## 0. 카탈로그 설정

In [ ]:
spark.sql("USE CATALOG P2T2")
print("✅ Catalog: P2T2")

## 1. 라이브러리 및 설정

In [ ]:

import openai
import json
import time
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Azure Key Vault에서 API 키 로드
OPENAI_API_KEY = dbutils.secrets.get(scope="key-vault-scope", key="openai-api-key")
OPENAI_ENDPOINT = dbutils.secrets.get(scope="key-vault-scope", key="openai-endpoint")

# Azure OpenAI 클라이언트
client = openai.AzureOpenAI(
    azure_endpoint=OPENAI_ENDPOINT,
    api_key=OPENAI_API_KEY,
    api_version="2025-01-01-preview"
)

MODEL = "gpt-51-deploy"  # Azure OpenAI 배포명
MAX_TOKENS = 1500
TEMPERATURE = 0.3  # 의료: 낮은 temperature로 일관성 확보

print(f"🔧 Azure OpenAI: Model={MODEL}, MaxTokens={MAX_TOKENS}")

## 2. SOAP 프롬프트 템플릿

In [ ]:

SOAP_PROMPT = """
You are an experienced emergency medicine AI assistant. Generate a SOAP note
for the following patient data.

## Patient Information
- Patient ID: {patient_id}
- Diagnoses: {diagnoses}
- Medications: {medications}

## Vital Signs Summary (Silver → Gold aggregated)
- Heart Rate: {avg_heart_rate} bpm
- Systolic BP: {avg_systolic_bp} mmHg
- Diastolic BP: {avg_diastolic_bp} mmHg
- SpO2: {avg_spo2}%
- Temperature: {avg_temperature}°C
- Respiratory Rate: {avg_respiratory_rate} /min
- Risk Score: {max_risk_score}/1.0

## Output Format
### S (Subjective)
### O (Objective)
### A (Assessment)
### P (Plan)
"""

print("✅ SOAP 프롬프트 정의 완료")

## 3. 추론 실행

In [ ]:

def run_soap_inference(patient_row):
    """
    단일 환자에 대해 SOAP 노트를 생성합니다.
    
    Args:
        patient_row: Gold Layer patient_clinical_summary Row
    """
    prompt = SOAP_PROMPT.format(
        patient_id=patient_row["patient_id"],
        diagnoses=patient_row.get("diagnoses", "N/A"),
        medications=patient_row.get("medications", "N/A"),
        avg_heart_rate=patient_row["avg_heart_rate"],
        avg_systolic_bp=patient_row["avg_systolic_bp"],
        avg_diastolic_bp=patient_row.get("avg_diastolic_bp", "N/A"),
        avg_spo2=patient_row["avg_spo2"],
        avg_temperature=patient_row.get("avg_temperature", "N/A"),
        avg_respiratory_rate=patient_row.get("avg_respiratory_rate", "N/A"),
        max_risk_score=patient_row["max_risk_score"],
    )
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "You are a clinical decision support AI for emergency medicine."},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
        )
        soap_note = response.choices[0].message.content or "[Content filtered]"
        tokens = response.usage.total_tokens if response.usage else 0
        return {
            "patient_id": patient_row["patient_id"],
            "soap_note": soap_note,
            "model_version": MODEL,
            "tokens_used": str(tokens),
        }
    except Exception as e:
        print(f"⚠️ API 오류 ({patient_row['patient_id']}): {str(e)}")
        return {
            "patient_id": patient_row["patient_id"],
            "soap_note": f"Error: {str(e)}",
            "model_version": MODEL,
            "tokens_used": "0",
        }

print("✅ SOAP 추론 함수 준비 완료")

## 4. 배치 추론 및 저장

In [ ]:

def batch_soap_inference(limit=100, delay=0.5):
    """
    Gold Layer 환자 데이터에 대해 배치 SOAP 추론 후 결과를 저장합니다.
    """
    df_patients = spark.sql(f"""
        SELECT * FROM P2T2.gold.patient_clinical_summary
        ORDER BY max_risk_score DESC
        LIMIT {limit}
    """)
    
    patients = df_patients.collect()
    results = []
    
    print(f"🚀 배치 SOAP 추론 시작: {len(patients)}명")
    
    for idx, patient in enumerate(patients):
        result = run_soap_inference(patient)
        results.append(result)
        if (idx + 1) % 10 == 0:
            print(f"   진행: {idx + 1}/{len(patients)}")
        time.sleep(delay)
    
    df_results = spark.createDataFrame(results)
    df_results = df_results.withColumn("created_at", F.current_timestamp())
    
    df_results.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("P2T2.ai_results.openai_soap_notes")
    
    print(f"\n✅ SOAP 추론 완료: {len(results)}건 → P2T2.ai_results.openai_soap_notes")

batch_soap_inference(limit=100, delay=1.0)